# fairgraph

## 

## Installation

In [8]:
! pip install fairgraph
! pip install tabulate

## Authentication

You will need to retrieve an EBRAINS authentication token. There are several ways to do this. For example you can log in to https://editor.kg.ebrains.eu and then click "Copy token to clipboard" in the user menu, top right.

Once you have a token, paste it here:

In [1]:
token = "..."

You can then create a client for communicating with the KG:

In [2]:
from fairgraph import KGClient

kg_client = KGClient(token, host="core.kg.ebrains.eu")

## Retrieving nodes

We'll use the software "PyNN" as our example. Let's get a list of all versions of PyNN in the KG:

In [3]:
import fairgraph.openminds.core as omcore
omcore.set_error_handling(None)

software_versions = omcore.SoftwareVersion.list(kg_client, name="PyNN")
software_versions.sort(key=lambda ver: ver.release_date)

Mismatched types: <class 'fairgraph.openminds.core.miscellaneous.comment.Comment'> <> ['https://openminds.ebrains.eu/publications/LearningResource']
Mismatched types: <class 'fairgraph.openminds.core.miscellaneous.comment.Comment'> <> ['https://openminds.ebrains.eu/publications/LearningResource']
Mismatched types: <class 'fairgraph.openminds.core.miscellaneous.comment.Comment'> <> ['https://openminds.ebrains.eu/publications/LearningResource']


In [18]:
from IPython.display import display, HTML 
from tabulate import tabulate

data = [
    {"name": ver.name, "version": ver.version_identifier, "release_date": ver.release_date} 
    for ver in software_versions
]
display(HTML(tabulate(data, headers="keys", tablefmt="html")))


name,version,release_date
PyNN,0.9.5,2019-12-05 00:00:00
PyNN,0.9.6,2020-12-17 00:00:00
PyNN,0.10.0,2021-12-06 00:00:00
PyNN,0.10.1,2022-10-13 00:00:00
PyNN,0.11.0,2023-05-23 00:00:00


In [6]:
latest_version = software_versions[-1]
latest_version.show()

id                         https://kg.ebrains.eu/api/instances/95281d91-fae4-41b3-a23c-ac2c4078e86a
space                      software
type                       https://openminds.ebrains.eu/core/SoftwareVersion
name                       PyNN
alias                      PyNN-0.11.0
accessibility              KGProxy([<class 'fairgraph.openminds.controlledterms.product_accessibility.ProductAccessibility'>], 'https://kg.ebrains.eu/api/instances/b2ff7a47-b349-48d7-8ce4-cf51868675f1')
application_categories     KGProxy([<class 'fairgraph.openminds.controlledterms.software_application_category.SoftwareApplicationCategory'>], 'https://kg.ebrains.eu/api/instances/4355ec3e-8d53-4c59-ba65-931132a78951')
copyright                  Copyright(years='2006')
custodians                 None
description                PyNN (pronounced 'pine') is a simulator-independent language for building neuronal network models.

                           In other words, you can write the code for a model once, u

## Following links in the graph

There is a **lot** of information here. You'll notice that some of the properties just contain text, like `version_identifier: 0.11.0`, but others contain a `KGProxy`. This represents an "edge" in the graph, i.e. a link to another metadata node.

To follow these links, we must "resolve" them:

In [7]:
tutorials = [
    lr.resolve(kg_client) 
    for lr in latest_version.learning_resources
]

Rather than showing _all_ the metadata this time, we'll just select certain fields (also called "properties"). 
To see a list of all the fields for a learning resource, use:

In [8]:
[f.name for f in tutorials[0].fields]

['name',
 'iri',
 'about',
 'abstract',
 'authors',
 'cited_publications',
 'copyright',
 'creation_date',
 'custodians',
 'digital_identifier',
 'editors',
 'educational_level',
 'funding',
 'keywords',
 'learning_outcome',
 'license',
 'modification_date',
 'order',
 'prerequisite',
 'publication_date',
 'publisher',
 'required_time',
 'topic',
 'type',
 'version_identifier']

In [9]:
data = [
    {"name": tut.name, "iri": tut.iri, "learning outcome": tut.learning_outcome, "prerequisite": tut.prerequisite}
    for tut in tutorials
]
HTML(tabulate(data, headers="keys", tablefmt="html"))

name,iri,learning outcome,prerequisite
Building and simulating a simple model,https://youtu.be/zBLNfJiEvRc,"In this tutorial, you will learn how to build a simple network of integrate-and-fire neurons using PyNN, how to run simulation experiments with this network using different simulators, and how to visualize the data generated by these experiments.",a basic knowledge of neuroscience (high-school level or above); basic familiarity with the Python programming language
Installing PyNN on a Linux system,https://youtu.be/BZB7xKUK8Vc,"In this tutorial, you will learn how to install PyNN, together with the Brian 2, NEST and NEURON simulators, on Linux.",a basic knowledge of neuroscience (high-school level or above); basic familiarity with the Python programming language


## Pre-fetching links

Following links one at a time can be quite slow. If you know in advance which links you're interested in, 
you can specify them at the time of the query.

Let's re-fetch the latest version of PyNN, but follow some of the links in advance:

In [10]:
latest_version_with_links = omcore.SoftwareVersion.from_id(latest_version.id, kg_client,
                                                           follow_links={"accessibility": {}, "licenses": {}})

Mismatched types: <class 'fairgraph.openminds.core.miscellaneous.comment.Comment'> <> ['https://openminds.ebrains.eu/publications/LearningResource']


In [11]:
print(f"Accessibility (simple query): {latest_version.accessibility}\n")
print(f"Accessibility (follow_links): {latest_version_with_links.accessibility}\n")
print(f"License (simple query): {latest_version.licenses}\n")
print(f"License (follow_links): {latest_version_with_links.licenses}")

Accessibility (simple query): KGProxy([<class 'fairgraph.openminds.controlledterms.product_accessibility.ProductAccessibility'>], 'https://kg.ebrains.eu/api/instances/b2ff7a47-b349-48d7-8ce4-cf51868675f1')

Accessibility (follow_links): ProductAccessibility(name='free access', definition="With 'free access' selected, data and metadata are both released and become immediately available without any access restrictions.", space=None, id=https://kg.ebrains.eu/api/instances/b2ff7a47-b349-48d7-8ce4-cf51868675f1)

License (simple query): KGProxy([<class 'fairgraph.openminds.core.data.license.License'>], 'https://kg.ebrains.eu/api/instances/cc878c03-9838-4f09-a856-66c831775430')

License (follow_links): License(name='CeCILL Free Software License Agreement v2.1', alias='CECILL-2.1', legal_code=IRI(https://spdx.org/licenses/CECILL-2.1.html), webpages='https://opensource.org/licenses/CECILL-2.1', is_applied_to=[KGProxy([<class 'fairgraph.openminds.core.products.software_version.SoftwareVersion'>]

You can also follow multiple levels of links, e.g.::

In [12]:
follow_links = {
    "developers": {
        "affiliations": {
            "member_of": {}
        }, 
        "digital_identifiers": {}
    }
}
latest_version_with_developers = omcore.SoftwareVersion.from_id(latest_version.id, kg_client, follow_links=follow_links, scope="released")

Mismatched types: <class 'fairgraph.openminds.core.products.model_version.ModelVersion'> <> ['https://openminds.ebrains.eu/publications/LearningResource']
Mismatched types: <class 'fairgraph.openminds.core.miscellaneous.comment.Comment'> <> ['https://openminds.ebrains.eu/publications/LearningResource']


In [13]:
latest_version_with_developers.developers[0].show()

id                    https://kg.ebrains.eu/api/instances/bd554312-9829-4148-8803-cb873d0b32f9
space                 None
type                  https://openminds.ebrains.eu/core/Person
affiliations          Affiliation(member_of=Organization(name='Paris-Saclay Institute of Neuroscience, Centre National de la Recherche Scientifique, Université Paris-Saclay, Saclay, France', alias='NeuroPSI', has_parents=KGProxy([<class 'fairgraph.openminds.core.actors.organization.Organization'>], 'https://kg.ebrains.eu/api/instances/31259b06-91d0-4ad8-acfd-303fc9ed613b'), homepage=IRI(https://neuropsi.cnrs.fr/), has_children=KGProxy([<class 'fairgraph.openminds.core.actors.organization.Organization'>], 'https://kg.ebrains.eu/api/instances/31259b06-91d0-4ad8-acfd-303fc9ed613b'), space=None, id=https://kg.ebrains.eu/api/instances/7bdf4340-c718-45ea-9912-41079799dfd3), start_date=datetime.datetime(2019, 1, 1, 0, 0))
alternate_names       None
associated_accounts   None
contact_information   None
digital_i

## A few hints and tips

If a property contains a link to a single node, it will contain just that node. If it contains a link to multiple nodes, it will contain a list of the nodes. If it doesn't link to anything, the property will contain `None`. If you're trying to automate things, it can be painful to have to check every time whether you have an object, a list, or None. For this, fairgraph provides the utility function `as_list()`:

In [14]:
from fairgraph.utility import as_list

as_list(42)

[42]

In [15]:
as_list([42])

[42]

In [16]:
as_list(None)

[]

Using this, you're always safe to write `for item in as_list(kg_objects):`

### Getting help

The Python docstrings for each class and method contain a lot of helpful information:

In [28]:
from fairgraph.openminds.core import Software

help(Software)

Help on class Software in module fairgraph.openminds.core.products.software:

class Software(fairgraph.kgobject.KGObject)
 |  Software(name=None, alias=None, custodians=None, description=None, developers=None, digital_identifier=None, versions=None, homepage=None, how_to_cite=None, comments=None, is_part_of=None, learning_resources=None, id=None, data=None, space=None, scope=None)
 |  
 |  Structured information on a software tool (concept level).
 |  
 |  Args
 |  ----
 |  name : str
 |      Whole, non-abbreviated name of the software.
 |  alias : str
 |      Shortened or fully abbreviated name of the software.
 |  custodians : ~fairgraph.openminds.core.actors.consortium.Consortium, ~fairgraph.openminds.core.actors.organization.Organization, ~fairgraph.openminds.core.actors.person.Person
 |      The 'custodian' is a legal person who is responsible for the content and quality of the data, metadata, and/or code of a research product.
 |  description : str
 |      Longer statement or acc

### Statistics

To retrieve the number of nodes that match a given query, but without retrieving all of the data, use the `count()` method. This can be much quicker if there are a lot of nodes.

In [17]:
mmigliore = omcore.Person.list(kg_client, family_name="Migliore", given_name="Michele")[0]
n_mmigliore_models = omcore.Model.count(kg_client, custodian=mmigliore)
print(n_mmigliore_models)

Mismatched types: <class 'fairgraph.openminds.core.products.model_version.ModelVersion'> <> ['https://openminds.ebrains.eu/publications/ScholarlyArticle']


225


### openMINDS modules

The KG uses openMINDS schemas, which are divided into different modules. In fairgraph, each of these is represented by a Python module, and each schema by a Python class. To get a list of modules:

In [22]:
import fairgraph.openminds
[mod for mod in dir(fairgraph.openminds) if not mod.startswith("__")]

['chemicals',
 'computation',
 'controlledterms',
 'core',
 'ephys',
 'publications',
 'sands',
 'specimenprep',
 'stimulation']

To get a list of the classes in each module, use the `list_kg_classes()` function:

In [26]:
import fairgraph.openminds.controlledterms as terms

classes = terms.list_kg_classes()
print([cls.__name__ for cls in classes])

['ActionStatusType', 'AgeCategory', 'AnalysisTechnique', 'AnatomicalAxesOrientation', 'AnatomicalIdentificationType', 'AnatomicalPlane', 'AnnotationCriteriaType', 'AnnotationType', 'AtlasType', 'AuditoryStimulusType', 'BiologicalOrder', 'BiologicalSex', 'BreedingType', 'CellCultureType', 'CellType', 'ChemicalMixtureType', 'Colormap', 'ContributionType', 'CranialWindowConstructionType', 'CranialWindowReinforcementType', 'CriteriaQualityType', 'DataType', 'DeviceType', 'DifferenceMeasure', 'Disease', 'DiseaseModel', 'EducationalLevel', 'ElectricalStimulusType', 'EthicsAssessment', 'ExperimentalApproach', 'FileBundleGrouping', 'FileRepositoryType', 'FileUsageRole', 'GeneticStrainType', 'GustatoryStimulusType', 'Handedness', 'Language', 'Laterality', 'LearningResourceType', 'MeasuredQuantity', 'MetaDataModelType', 'ModelAbstractionLevel', 'ModelScope', 'MolecularEntity', 'OlfactoryStimulusType', 'OperatingDevice', 'OperatingSystem', 'OpticalStimulusType', 'Organ', 'OrganismSubstance', 'Org

## Adding new metadata to the Knowledge Graph